In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\devin\AI-agriculture-yield-prediction\Datasets\rice_data_outlier_removed.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\devin\\AI-agriculture-yield-prediction\\Datasets\\rice_data_outlier_removed.csv'

In [ ]:
x = df[['Year' , 'RICE AREA (1000 ha)' , 'RICE PRODUCTION (1000 tons)', 'State_en']]
y = df['RICE YIELD (Kg per ha)']

In [ ]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x ,y ,test_size = 0.2 , random_state = 42)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train_scaled = ss.fit_transform(x_train)
x_test_scaled = ss.transform(x_test)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf, x_train_scaled, y_train, cv=5)
print(scores.mean() * 100)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
rf = RandomForestRegressor(n_estimators=90, max_depth=20, random_state=42)
rf.fit(x_train_scaled,y_train)

In [ ]:
rf.score(x_test_scaled,y_test)*100

In [ ]:
rf.score(x_train_scaled,y_train)*100

In [ ]:
import joblib
import os
os.makedirs("models", exist_ok=True)
joblib.dump(rf, 'models/rf_model.pkl')
joblib.dump(ss, 'models/scaler.pkl')


In [ ]:
y_pred = rf.predict(x_test_scaled)
pred_df = pd.DataFrame({'Actual': y_test,'Predicted': y_pred})
pred_df.to_csv(r'C:\Users\devin\AI-agriculture-yield-prediction\Models\test_predictions.csv', index=False)